# The discrete time Kuramoto model
We create a class for this (Model 1) with the intention of getting the data for the images of the paper

In [2]:
import numpy as np
import pandas as pd
from pandas import Series
# import pathlib2 as Path
import matplotlib.pyplot as plt
import math
import time
import statistics
import random
import time
import os.path
import shutil
import sys

from sklearn.linear_model import LinearRegression

from scipy.stats import invgamma
from scipy.stats import norm

from scipy.sparse import csc_array


# import torch


In [3]:
class KuramotoModel:                    #Model 1, using th values in 0,2pi
    
    def __init__(self, edge_list, alpha = 1.0):                    #intput is a list of lists of edges
        self.N = len(edge_list)                       
        self.edge_list = edge_list
        ed_left, ed_right = list_of_edges_to_left_right(edge_list)
        CN = len(ed_left)
        #Compressed Sparce Column format. It is faster than csr for dot
        self.edge_list_error = False
        if max(ed_left)>= self.N or max(ed_right)>= self.N:
            print('Error inside __init__ Kuramoto: \n edge index larger than N', self.N, CN//self.N, max(ed_left), max(ed_right))
            self.edge_list_error = True
        else:
            self.matrix = csc_array((np.ones(CN), (ed_left, ed_right)), shape=(self.N, self.N))  
        
            # basic graph statistics
            self.edge_size_list = np.array([len(eds) for eds in edge_list])
            self.C = np.around(self.edge_size_list.mean(),2)
            self.max_degree = max(self.edge_size_list)
            self.w_vector = self.edge_size_list / self.C                 # this is fixed
            self.M2 = np.mean(self.w_vector*self.w_vector)                 # 2nd Moment of distribution of w
            self.M3 = np.mean(self.w_vector*self.w_vector*self.w_vector)   # 3rd Moment of distribution of w
            self.M4 = np.mean(self.w_vector*self.w_vector*self.w_vector*self.w_vector)
            self.sigma = (0.5*self.M2/self.N)**0.5                         #1D-std of choosing N vectors from circle 

            self.until_r = 6 * self.sigma               # for until_r uses 6 * self.sigma. The probability of going beyond is ??


            # Parameter for the dynamics, 
            # self.alpha    ... sets  alpha                                          
            # self.kbeta    ... kbeta is the estimated coeficient in the complex-square+noise Markov Chain
            self.set_alpha(alpha)

            #States, which change in dynamical system 
            #         self.th_vector                       ..... creates the vector of states 
            #         self.cos_vector                       # these are the state coordinates in circle
            #         self.sin_vector 
            #         self.c 
            #         self.s 
            #         self.r, 
            self.set_uniform()

        


    # chose states randomly from the uniform distribution
    def set_uniform(self):
        self.th_vector = np.random.uniform(0, 2*np.pi, size = self.N)
        self.time = 0
        
        self.update_r_theta()

        self.r_historic = [self.r]
        self.theta_historic = [self.theta]

 

    # update values from new th_vector
    def update_r_theta(self):
        self.cos_vector = np.cos(self.th_vector)                      # these are the state coordinates in circle
        self.sin_vector = np.sin(self.th_vector)
        self.c = np.dot(self.cos_vector, self.w_vector)/self.N
        self.s = np.dot(self.sin_vector, self.w_vector)/self.N
        self.r, self.theta = length_angle((self.c, self.s))
        

    def set_alpha(self, alpha):
        self.alpha = alpha                                            
        self.kbeta = np.mean(self.w_vector**3 * np.exp(- self.w_vector * self.alpha**2 / (4*self.C)))
        self.sbeta = (np.mean((self.w_vector**2 * np.exp(- self.w_vector * self.alpha**2 / (4*self.C))-
                               self.w_vector*self.kbeta/self.M2)**2))**0.5
        
    def estimated_expected_time(self):
        sample_size_for_simulation = 100000
        self.k_coeficient = self.alpha**2 * self.kbeta * (1+4/(self.alpha * self.M2)) /8
        self.estimated_spark_time = cuadratic_escape_function(self.k_coeficient * self.sigma, sample_size_for_simulation)
        
        return self.estimated_spark_time
    
    
        
    # DYNAMICS
    # calculates one step of the Kuramoto function  z <-  2 z + (alpha/C) sum( sin(z_j - z))
    def one_step(self): 
                    
        V_cos = self.matrix.dot(self.cos_vector)    ##   timely steps
        V_sin = self.matrix.dot(self.sin_vector)    ##
        
        self.th_vector *= 2 
        self.th_vector += (self.alpha / self.C) * (V_sin * self.cos_vector - V_cos * self.sin_vector)
        self.th_vector = self.th_vector % (2 * np.pi)
        
        self.update_r_theta()
        self.r_historic.append(self.r)
        self.theta_historic.append(self.theta)

        return self.r
                

    # calculates one step of the Kuramoto function assuming V = r * di
    def one_step_r(self,r): 
                            
        self.th_vector = (2 * self.th_vector) % (2 * np.pi)
        self.th_vector += self.alpha * self.w_vector * r *  self.sin_vector
        
        self.update_r_theta()
        self.r_historic.append(self.r)
        self.theta_historic.append(self.theta)

        return self.r

    
    
    # runs model from uniform (unless set False) until either r eaches until_r, or there has been until_max_t steps
    # return the number of steps taken
    # records history of r and thetas
    def run_until_sync(self, until_max_t, from_uniform = True):

        
        if from_uniform:
            self.set_uniform()
        
        for t in range(until_max_t):
            if self.r > self.until_r: 
                return t


            ######### ONE STEP
            self.one_step()

        # if no early return, returns until_max_t
        return until_max_t
    

    
    
    # runs run_until_sync over and over until total_steps_to_run have been run. 
    # Then calculates expeted value of spark
    def expected_stages_to_sync(self, total_steps_to_run):
        
        
        # for printing rotating bar
        horizontal = False                      
        steps_to_print = total_steps_to_run//20
        
        #for saving history before spark, in and out values
        self.r_pre_in_historic =[]              
        self.r_pre_out_historic =[]
        self.theta_pre_in_historic =[]
        self.theta_pre_out_historic =[]
        steps_to_sync = 5                       # does not take into account steps_to_sync before sync


        runs_left = total_steps_to_run

        # for recording the times for each run_until_sync
        list_of_Tes = []
        
        # START RUNS
        self.set_uniform()

        while runs_left > 0:
            t0 = self.run_until_sync(runs_left)       # RUNS happen here
            list_of_Tes.append(t0)

            rl = runs_left
            runs_left -= t0 +1

            # recoding activity before syncing
            self.r_pre_in_historic += self.r_historic[:-steps_to_sync]
            self.r_pre_out_historic += self.r_historic[1:-(steps_to_sync-1)]
            self.theta_pre_in_historic += self.theta_historic[:-steps_to_sync]
            self.theta_pre_out_historic += self.theta_historic[1:-steps_to_sync+1]
 
            #### printing | and - to show activity 
#             if (rl//steps_to_print)-(runs_left//steps_to_print) > 0: 
#                 print('\b', end='')
#                 if horizontal: print('-', end='')        
#                 else: print('|', end='')
#                 horizontal = not horizontal
#         print('\b', end='')

        self.sim_escape, self.preciscion_esc, self.climb = expected_value_constant_plus_exp(list_of_Tes)
        self.number_of_iterates = total_steps_to_run
        
        return list_of_Tes


    def regression_data(self):
        data_Rfunc = Data_for_R()
        data_Rfunc.initialize_from_model(self)
        data_Rfunc.calculate_values_light_cuadratic_coef()
        
        self.coefLR = data_Rfunc.coefLR
        self.rss = data_Rfunc.rss
        
        self.dgp = data_Rfunc.dgp
        
        return self.coefLR


    
    
#     def vizualize(self):

Useful functions

In [4]:
# decompososes pair (a,b) representing complex number r e^theta
def length_angle(p):
    return np.absolute(p[0]+p[1]*1j), np.angle(p[0]+p[1]*1j)


# We now look at the escape function

In [5]:
# the input is a list or time with distribution constant + small error + geometrical with parameter lambda
# the output is lamdba, the std for the estimation of lambda, and the constant

# doesn't count the last value

def expected_value_constant_plus_exp(t_values):
    t_values = np.array(t_values)

    if len(t_values) == 0:
        return 0, 0, 0
    
    var_tot = np.var(t_values)
    std_tot = var_tot**0.5
    mean_tot = np.mean(t_values)
    constant = np.min(t_values)
    
    bound = 2 + mean_tot//5      # ad-hoc bound. It how much after the counstant we will cut_off
    
    var_rest = var_tot
    std_rest = std_tot
    mean_rest = mean_tot
    
    done = False
    
    counter = 5
    old_cut_off = 0
    cut_t_values = t_values
    cut_off = int(constant + bound) 
    
    while not done:
        cut_t_values = cut_t_values[cut_t_values >= cut_off]
        n = len(cut_t_values)
        if n == 0:
            return mean_rest, -1, constant

        mean_rest = np.mean(cut_t_values) - cut_off
        constant = np.max([mean_tot - mean_rest, 0])
        counter -= 1
        cut_off = int(constant + bound) 
        
        done = (cut_off <= old_cut_off) or counter == 0
        old_cut_off = cut_off

    var_rest = np.var(cut_t_values)
#     std_climb = np.max([var_tot - var_rest, 0])**0.5
    std_mean_rest = mean_rest/n**0.5            # calculated using Gamma prior for 1/mean_rest
    
    return mean_rest, std_mean_rest, constant




#### gives the expected value for time to escape of the function x**2 with gauss-sigma
def cuadratic_escape_function_list(sig, sample_size, until_r = 10):
    t_minumum = 1
    runs_left = sample_size -1
    list_of_Tes = []
    
    until_r2 = until_r ** 2
    
    ran_vec_c = np.random.normal(0,sig, sample_size)
    ran_vec_s = np.random.normal(0,sig, sample_size)
    
    while runs_left >= 0:
        t0= -1
        r2 = 0                                    ## represets r squared
        while runs_left >= 0 and r2 < until_r2:
            cN = r2 + ran_vec_c[runs_left]
            sN = ran_vec_s[runs_left]
            r2 = (cN**2+sN**2)
            t0 +=1
            runs_left -= 1

        if (r2 >= until_r2): 
            list_of_Tes.append(t0)

    return list_of_Tes



#### gives the expected value for time to escape of the function x**2 with gauss-sigma
def cuadratic_escape_function(sig, sample_size):
    
    list_of_Tes = cuadratic_escape_function_list(sig, sample_size)
    exp_time, _, _ = expected_value_constant_plus_exp(list_of_Tes)
        
    return exp_time



# Genertaes random graphs to initialize models

In [6]:
# Time: run with 100000 nodes with 1000 edges each took 54 minutes
def generate_random_graph(degree_vector, verbose = False):
    N = len(degree_vector)

    # Managing Errors
    sdv  = sum(degree_vector)
    # Odd case: s has to be even. If it isn't, we add one edge to node 0
    if  sdv % 2 == 1: degree_vector[0] += 1
    # Maximum too big
    if max(degree_vector) >= N:
        print("Some degrees are greater than N")
        degree_vector = np.clip(degree_vector, 0, N-1)
    
    max_steps_2 = 10                                # maximum number of setps in Stage 2
    
    list_of_edges= [set({}) for _ in range(N)]         # edges we have allocated to each node
    num_of_edges = np.zeros(N, dtype = int)            # edges we have allocated so far
    remain_degree_vector = np.copy(degree_vector)      # edges one still needs to find
    s = int(sum(remain_degree_vector))                 # total number missing
    
    all_edge_pairs = []                               # edges that have been allocated, and may have or not been removed

    
    
    

    dic = {}
    indx = int(0)
    for node_j, dv in enumerate(degree_vector):
        for _ in range(dv):
            dic[indx] = node_j
            indx += 1

    
    # Stage one --- main edge creation
    almost_done = False
    while not almost_done:
        indices_left = list(dic)
        random.shuffle(indices_left)
        num_left = len(indices_left)
        if verbose: print(f"Generating random graph: Step 1: Nodes left {num_left}")
        for t in range(0,len(indices_left),2):
            i = indices_left[t]
            j = indices_left[t+1]
            if i in dic and j in dic:
                node_i = dic[i]
                node_j = dic[j]
                good_pair = (node_i != node_j) and (node_j not in list_of_edges[node_i])
                if good_pair:
                    list_of_edges[node_i].add(node_j)
                    list_of_edges[node_j].add(node_i)
                    num_of_edges[node_i] += 1
                    num_of_edges[node_j] += 1
                    all_edge_pairs.append((i,node_i,j,node_j))      # only one pair added
                    del dic[i]
                    del dic[j]
                    num_left -= 2


        almost_done = (num_left >= 0.7 * len(indices_left))
        
    # Stage two --- choose a pair node_i, node_j, and an edge(na,nb) and replace the edge for two new in the pair.
    for step in range(max_steps_2):
        indices_left = list(dic)
        random.shuffle(indices_left)
        num_left = len(indices_left)
        if verbose: print(f"Generating random graph: Step 2.{step}: Nodes left {num_left}")
        for t in range(0,len(indices_left),2):
            i = indices_left[t]
            j = indices_left[t+1]
            if i in dic and j in dic:
                node_i = dic[i]
                node_j = dic[j]
                good_pair = (node_i != node_j) and (node_j not in list_of_edges[node_i])
                if good_pair:
                    list_of_edges[node_i].add(node_j)
                    list_of_edges[node_j].add(node_i)
                    num_of_edges[node_i] += 1
                    num_of_edges[node_j] += 1
                    all_edge_pairs.append((i,node_i,j,node_j))      # only one pair added
                    del dic[i]
                    del dic[j]
                    num_left -= 2
                else:
                    for i_ed in range(len(all_edge_pairs)):
                        ed = all_edge_pairs[i_ed]
                        a, na, b, nb = ed
                        if na in list_of_edges[nb]:
                            if node_i not in list_of_edges[na]:
                                if node_j not in list_of_edges[nb]:
                                    if (node_i != na) and (node_j != nb):
                                        list_of_edges[na].remove(nb)
                                        list_of_edges[nb].remove(na)
                                        list_of_edges[na].add(node_i)
                                        list_of_edges[node_i].add(na)
                                        list_of_edges[nb].add(node_j)
                                        list_of_edges[node_j].add(nb)
                                        num_of_edges[node_i] += 1
                                        num_of_edges[node_j] += 1
                                        del dic[i]
                                        del dic[j]
                                        all_edge_pairs.pop(i_ed)
                                        done_ij=True
                                        break
        if not dic:
            break

    if dic:
        print(f"{sum(dic.values())} edges couldn't be matched")
                                    
    return list_of_edges


def list_of_edges_to_left_right(l_of_e):
    left = []
    right = []
    for node_i, ld in enumerate(l_of_e):
        for node_j in ld:
            left.append(node_i)
            right.append(node_j)
            
    return left, right



# returns int vector with the same sum as float vec
def round_vector(float_vec):
    decimals = float_vec % 1
    
    decimals.sort()
    m = decimals[len(decimals) - int(decimals.sum())]
        
    # we want int(decimals.sum()) numbers will be rounded up
    # use m as dividing line to decide whether to round up or down   
    round_vector = ((float_vec + 1 - m) // 1).astype(int)

    return(round_vector)

        
def generate_random_model(float_degree_vector):
    int_degree_vector = round_vector(float_degree_vector)
    edg_lis = generate_random_graph(int_degree_vector)
    return KuramotoModel(edg_lis)



def initialize_Model_inverse_gamma(N, C, g, minW=0.1, maxW=10):
    epsilon = 0.000000001
    min_quantile = invgamma.cdf(minW, g-1, loc=0, scale= g-2)
    max_quantile = invgamma.cdf(maxW, g-1, loc=0, scale= g-2)
    quantile_list = np.arange(min_quantile, max_quantile+epsilon, (max_quantile-min_quantile)/(N-1))
    
    wv = np.array([invgamma.ppf(q, g-1, loc=0, scale= g-2) for q in quantile_list])
    wv = wv / wv.mean()

    mod = generate_random_model(wv * C)
    mod.gamma = g
    mod.maxW = maxW
    
    mod.initialization = 'initialize_Model_inverse_gamma'
    
    return mod


def initialize_Model_truncated_inverse_gamma(N, C, gamma, maxW=10):
    wv = w_truncated_inverse_gamma(N, gamma, maxW)

    mod = generate_random_model(wv * C)
    mod.gamma = gamma
    mod.maxW = maxW
    mod.initialization = 'initialize_Model_truncated_inverse_gamma'
    
    return mod

# generates w_vector of size N with distribution inverse_gamma(g-1,scale) (power law gamma), truncated at maxW
# w_vector has mean 1, and scale is chosen so that this happens
def w_truncated_inverse_gamma(N, gamma, maxW):
    epsilon = 0.000000001
    
    scale = gamma - 2    # this would be the right scale if it wasn't truncated. It's a lower bouned
    
    new_mean = 0
    #Find de scale so that the mean is close to 1
    counter = 0
    while new_mean < 1 - epsilon:
        new_mean = invgamma.cdf(maxW, gamma-2, scale=scale)*scale/(gamma-2)
        scale = scale / new_mean
        counter +=1
        if counter > 100:
            print("more than 100 iterates in w_truncated_inverse_gamma")
            break
        
        
    maxq = invgamma.cdf(maxW, gamma-1, scale=scale)
    minq = 0.5 * maxq / N
    distq = maxq / N
    quantiles = np.arange(minq, maxq, distq)   # this is an evenly distributed array going upto maxq
    
    w_vec = np.array([invgamma.ppf(q, gamma-1,scale=scale) for q in quantiles])
    w_vec = w_vec/w_vec.mean()
        
    return w_vec

In [7]:
def save_invGamma_graph(extended_filename, N, C, gamma, maxW=10, verbose = False):    

    if verbose: print(f"Generating w-vector. N={N}, C={C}, g={gamma}")    
    w_vec = w_truncated_inverse_gamma(N, gamma, maxW)
    int_degree_vector = round_vector(w_vec * C)
    
    if verbose: print(f"Generating random-graph.")    
    edg_lis = generate_random_graph(int_degree_vector, verbose = verbose)        # this is the slow step
    
    with open(extended_filename, 'w') as f:
        for edges_of_node in edg_lis:
            f.write(" ".join(str(i) for i in edges_of_node) + " \n")
            
    return edg_lis

    
    
def read_invGamma_graph(extended_filename, N, C, gamma, maxW=10):
    edg_lis = []

    with open(extended_filename, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        edges_of_node = list(map(int, line.strip().split()))
        edg_lis.append(edges_of_node)

    return edg_lis

# generates list of edges for graph with inv_Gamma distribution
# it first check if the file exists to read it direclty. It not it creates and saves it.
def read_write_invGamma_graph(N, C, gamma, maxW=10, filename = "_", verbose = False):
    extended_filename = 'invGamma_graph' + filename + f"{N}_{C}_{gamma}_{maxW}.txt"

    if os.path.exists(extended_filename):
        edg_lis = read_invGamma_graph(extended_filename, N, C, gamma, maxW)
        new_N = len(edg_lis)
        new_inx =  max([max(el) for el in edg_lis])
        if new_N == N and new_inx<N:
            return edg_lis
        else:
            print(f"Error read_write_invGamma_graph: \n File {extended_filename} broken, creating new")
            shutil.copy(extended_filename, "broken_" + extended_filename)   # copy broken file

    edg_lis = save_invGamma_graph(extended_filename, N, C, gamma, maxW, verbose = verbose)
            
    return edg_lis


def init_Kuramoto_invGamma_file(N, C, gamma, maxW=10, filename = "_", verbose = False):

    edg_lis = read_write_invGamma_graph(N, C, gamma ,maxW, filename = filename, verbose = verbose)
        
    if verbose: print(f"Generating model from edge-list. N={N}, C={C}, g={gamma}")    
    mod =  KuramotoModel(edg_lis)        

    mod.gamma = gamma
    mod.maxW = maxW
    mod.initialization = 'init_Kuramoto_invGamma_file'
    mod.graph_filename = 'invGamma_graph' + filename + f"{N}_{C}_{gamma}_{maxW}.txt"
    
    return mod



def save_input_file(filename, Ns, Cs, alphas, gammas):
    with open(filename, 'w') as f:
        f.write(" ".join(str(i) for i in Ns) + " \n")
        f.write(" ".join(str(i) for i in Cs) + " \n")
        f.write(" ".join(str(i) for i in alphas) + " \n")
        f.write(" ".join(str(i) for i in gammas) + " \n")
        
def read_input_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        Ns = list(map(int, lines[0].strip().split()))   
        Cs = list(map(int, lines[1].strip().split()))
        alphas = list(map(int, lines[2].strip().split()))
        gammas = list(map(int, lines[3].strip().split()))

    return Ns, Cs, alphas, gammas

    

In [8]:
def estimated_expected_time(w_vec, alpha, C):
    sample_size_for_simulation = 100000

    N = len(w_vec)
    M2 = np.mean(w_vec*w_vec)
    sigma = (0.5*M2/N)**0.5
    kbeta = np.mean(w_vec**3 * np.exp(- w_vec * alpha**2 / (4*C)))
    k_coeficient = alpha**2 * kbeta * (1+4/(alpha * M2)) /8
    estimated_spark_time = cuadratic_escape_function(k_coeficient * sigma, sample_size_for_simulation)
    return estimated_spark_time

def K_and_M_time(w_vec, alpha, C):

    N = len(w_vec)
    M2 = np.mean(w_vec*w_vec)
    M3 = np.mean(w_vec*w_vec*w_vec)
    sigma = (0.5*M2/N)**0.5
    kbeta = np.mean(w_vec**3 * np.exp(- w_vec * alpha**2 / (4*C)))
    k_coeficient = alpha**2 * kbeta * (1+4/(alpha * M2)) /8
    
    K_alpha_delta_C = kbeta/ M3
    M_alpha_delta = M3 * M2**(0.5) * (1+4/(alpha * M2))
    return K_alpha_delta_C, M_alpha_delta



def estimated_expected_time_truncated_inverse_gamma(N, C, alpha, gamma, maxW):
    wve = w_truncated_inverse_gamma(N, gamma, maxW)
    return estimated_expected_time(wve, alpha, C)



# The next cell looks at how to get statistics from the historic on r-values

In [9]:
class Data_for_R:
    def __init__(self):
        self.num_bins = 6
        
    
    def initialize_from_model(self, m1):
        self.alpha = m1.alpha
        self.alpha_tilde= m1.alpha/m1.C
        self.beta = m1.alpha/ m1.C**0.5
        self.C = m1.C
                
        # df   contians historic information on r's and theta's
        self.df = pd.DataFrame(columns=['r0', 'theta0', 'r1', 'theta1'])
        self.df['r0'] = m1.r_pre_in_historic
        self.df['theta0'] = m1.theta_pre_in_historic
        self.df['r1'] = m1.r_pre_out_historic
        self.df['theta1'] = m1.theta_pre_out_historic

        
    def calculate_values(self):
        self.mean_r0 = self.df['r0'].mean()    # this should be M2 * (np.pi/N)**0.5 /2  
        self.mean_r1 = self.df['r1'].mean()
        self.std_r0 = self.df['r0'].std()
        self.std_r1 = self.df['r1'].std()

        # puts r's into bins
        self.max_bins = self.std_r0 * 5
        self.bin_width = self.max_bins / self.num_bins
        self.df['bin'] = (self.df['r0']*self.num_bins/ self.max_bins).astype(int)
        self.df = self.df[self.df['bin']<self.num_bins]

        # rotates to match angles. Assumes angle doubles
        self.df['tmint'] = self.df['theta1'] - 2*self.df['theta0']
        self.df['c'] = self.df['r1'] * np.cos(self.df['tmint'])
        self.df['s'] = self.df['r1'] * np.sin(self.df['tmint'])
#         self.df['c'] = self.df['r1'] * np.cos(2*np.pi* self.df['tmint'])
#         self.df['s'] = self.df['r1'] * np.sin(2*np.pi* self.df['tmint'])
        
        self.df['r0_squared_c'] = self.df['c'] * self.df['r0']**2
        self.df['r0_fourth'] = self.df['r0']**4

        
        self.dgp = self.df.groupby('bin').mean()
        self.dgp = self.dgp.drop(columns=['theta0', 'r1', 'theta1','tmint'])
        self.dgp['count'] = self.df.groupby('bin').count()['r0']
        self.dgp['c_std'] = self.df.groupby('bin').std()['c']
        self.dgp['s_std'] = self.df.groupby('bin').std()['s']
        self.dgp['r_out'] = (self.dgp['c']**2 + self.dgp['s']**2)**0.5

        self.mean_c = self.dgp['c'].mean()
        self.mean_s = self.dgp['s'].mean()
        self.max_c = self.dgp['c'].max()
        self.max_s = self.dgp['s'].max()
        self.c_std = self.dgp['c_std'].mean()
        self.s_std = self.dgp['s_std'].mean()

        self.dgp['regression_coef'] = self.dgp['r0_squared_c']/self.dgp['r0_fourth']
        
        
        
        

    def calculate_values_light_cuadratic_coef(self):
        self.mean_r0 = self.df['r0'].mean()     
        self.mean_r1 = self.df['r1'].mean()
        self.std_r0 = self.df['r0'].std()
        self.std_r1 = self.df['r1'].std()

        max_for_LR= self.std_r0 * 3
        
        self.df['c0'] = self.df['r0'] * np.cos(self.df['theta0'])
        self.df['s0'] = self.df['r0'] * np.sin(self.df['theta0'])   # thetas belong to 0-2pi
        self.df['c1'] = self.df['r1'] * np.cos(self.df['theta1'])
        self.df['s1'] = self.df['r1'] * np.sin(self.df['theta1'])

        self.df['c0c0'] = self.df['c0']**2
        self.df['s0s0'] = self.df['s0']**2
        self.df['c0s0'] = self.df['c0']*self.df['s0']


        
        
        ############  LINEAR REGRESSION 

        self.coefLR = None
        self.rss = None
        df_truncated = self.df[self.df['r0']<max_for_LR]

        Xdf = df_truncated.filter(['c0c0', 'c0s0', 's0s0'], axis=1)
        Ydf = df_truncated.filter(['c1','s1'], axis=1)

        lm =  LinearRegression()
        if len(Ydf)>1:
            lm.fit(Xdf,Ydf) 

            Y_pred = lm.predict(Xdf)
            Y_true = np.array(Ydf)
            rss2 = (sum(np.power((Y_pred - Y_true), 2))/len(Y_pred))**0.5
            self.rss = rss2.mean()

            self.cuad_regression_coef = lm.coef_
            self.cuad_regression_intercept = lm.intercept_

            self.coefLR = (self.cuad_regression_coef[0][0]-self.cuad_regression_coef[0][2]+self.cuad_regression_coef[1][1])/4

        

        ############  LINEAR REGRESSION by BIN

        # puts r's into bins
        self.max_bins = self.std_r0 * 5
        self.bin_width = self.max_bins / self.num_bins
        self.df['bin'] = (self.num_bins * self.df['r0']/ self.max_bins).astype(int)
        df_bins = self.df[self.df['bin']<self.num_bins]


        self.rss_bins = []
        self.coefLR_bins = []
        self.all_coefLR_bins = []
        
        self.dgp = df_bins.groupby('bin').mean()
        self.dgp = self.dgp.drop(columns=['theta0', 'r1', 'theta1'])
        self.dgp['count'] = df_bins.groupby('bin').count()['r0']

        self.dgp['regression_coef'] = 0
        
        Xdf = df_bins.filter(['c0c0', 'c0s0', 's0s0'], axis=1)
        Ydf = df_bins.filter(['c1','s1'], axis=1)

        
        for b in range(self.num_bins):
            Xdf_bin = Xdf[df_bins['bin'] ==  b]
            Ydf_bin = Ydf[df_bins['bin'] ==  b]

            if len(Ydf_bin)>1:
                lm_bin =  LinearRegression()
                lm_bin.fit(Xdf_bin,Ydf_bin) 

                Y_pred = lm_bin.predict(Xdf_bin)
                Y_true = np.array(Ydf_bin)
                self.rss_bins.append((sum(np.power((Y_pred - Y_true), 2))/len(Y_pred))**0.5)

                crc = lm_bin.coef_ 
                self.all_coefLR_bins.append(crc)
                cLR = (crc[0][0]-crc[0][2]+crc[1][1])/4
                self.coefLR_bins.append(cLR)

                self.dgp.loc[b,'regression_coef'] = cLR
        

        
        
        return lm
        
        
        
        



# Good ranges for simulation 

gamma 3-5

alpha 10-15

N     30000 - 100000

C     300   - 1000

maxW = 10

-------------

mid values:
estimated_expected_time_truncated_inverse_gamma(50000, 500, 12, 4, 10) = 19

# Extract data 

In [10]:
# input is a distribution setting
# the output is a dictionary with a lot of information 
def extract_data(disset: KuramotoModel, sample_size):

    disset.estimated_expected_time()
    limit_r_forced = force_sync(disset) 
    disset.until_r = disset.limit_r_forced / 2
    disset.expected_stages_to_sync(sample_size)
    disset.regression_data()
    
    features = ['N', 'C', 'alpha', 'gamma',
                'sim_escape', 'estimated_spark_time', 
                'k_coeficient', 'coefLR',
                'sigma','rss',
                'kbeta', 'M2', 'M3', 
                'preciscion_esc',
                'limit_r_forced',
                'std_r_forced',
                'number_of_iterates',
                'climb', 'maxW', 'sbeta','graph_filename']

    new_dic = {ky:disset.__dict__[ky] for ky in features}

    return new_dic




    
def force_sync(disset: KuramotoModel):
    alpha = disset.alpha
    C = disset.C
    infinity_r = 100
    
    r_min_fixed_point = 1/ disset.k_coeficient
    
    # disset.set_uniform()


        
    for _ in range(20):
        r = max(disset.r, r_min_fixed_point)
        disset.one_step_r(r)

    disset.run_until_sync(infinity_r, from_uniform = False)
    

    for _ in range(20):
        disset.one_step()

    disset.limit_r_forced = np.mean(disset.r_historic[-10:])
    disset.std_r_forced = np.std(disset.r_historic[-10:])
    
    return disset.limit_r_forced




def extract_and_save(sample_size, N, C, alphas, gamma, maxW=10, filename = "_"):
    
        disset = init_Kuramoto_invGamma_file(N , C , gamma)
        
        if disset.edge_list_error:
            print(f"Kuramoto for {N}, {C}, {gamma}, never created, exiiting")
            return None
        
        new_values = []
        
        for alpha in alphas:
                  
            disset.set_alpha(alpha)
            extended_filename = 'data_Model1' + filename + f"{sample_size}.csv"

            if os.path.exists(extended_filename):
                runs_info_df = pd.read_csv(extended_filename, index_col = 0)
            else:
                runs_info_df = pd.DataFrame(columns = ['N','C','alpha', 'gamma', 'maxW'])

            filtro = (np.around(runs_info_df['alpha'],3) == np.around(alpha,3)) 
            filtro = filtro  & (runs_info_df['C'] == disset.C) 
            filtro = filtro  & (np.around(runs_info_df['gamma'],3) == np.around(gamma, 3)) 
            filtro = filtro  & (runs_info_df['N'] == N) 
            filtro = filtro  & (runs_info_df['maxW'] == maxW) 


            if not filtro.any():
                info_dic = extract_data(disset, sample_size)

                if os.path.exists(extended_filename):
                    runs_info_df = pd.read_csv(extended_filename, index_col = 0)
                else:
                    runs_info_df = pd.DataFrame(columns = ['N','alpha', 'C','gamma', 'maxW'])

                runs_info_df = pd.concat([runs_info_df,pd.DataFrame.from_dict([info_dic])])
                runs_info_df.to_csv(extended_filename)
                new_values.append((N,C,alpha,gamma))

            
        return new_values



    
def merge_files_dataModel1(samplesize1, samplesize2, filename1 = "_", filename2 = "_", filename_out = "_added_" ):
    ss = samplesize1+samplesize2
    extended_filename1 = 'data_Model1' + filename1 + f"{samplesize1}.csv"
    extended_filename2 = 'data_Model1' + filename2 + f"{samplesize2}.csv"
    extended_filename_out = 'data_Model1' + filename_out + f"{ss}.csv"
    
    if os.path.exists(extended_filename_out):
        print(f"file {extended_filename_out} already exists, change value of filename_out.")
        return None
              
 #############   SORT BY graph_filename and alpha
              
    df1 = pd.read_csv(extended_filename1, index_col = 0)
    df2 = pd.read_csv(extended_filename2, index_col = 0)

    df1 = df1.sort_values(by=['N', 'C', 'alpha', 'gamma', 'graph_filename'])
    df2 = df2.sort_values(by=['N', 'C', 'alpha', 'gamma', 'graph_filename'])
              
    df2['sim_escape'] = (df1['sim_escape']*samplesize1 + df2['sim_escape']*samplesize2)/ ss
    df2['coefLR'] = (df1['coefLR']*samplesize1 + df2['coefLR']*samplesize2)/ ss
#     df2['rss'] = ??
#     df2['preciscion_esc'] = ??
    df2['number_of_iterates'] = ss
    
    df2.to_csv(extended_filename_out)
     
    

In [11]:
def figures_last_r(N, C, gamma, maxW, t_averages, T_stabilization):

    mod1 = init_Kuramoto_invGamma_file(N, C, gamma, maxW=maxW, verbose = True)   # initiate setting for the model
    
    alphas = np.concatenate((np.linspace(3,0.5, 26) , np.linspace(0.49, 0, 50)))     

    r_averages = []
    r_stds = []

    length_for_even_ods = min(50, t_averages-1)
    r_average_odds = []
    r_average_evens = []

    for a in alphas:
        mod1.set_alpha(2*np.pi * a)   # Mupltiplying by 2 pi because they use alpha for [0,1]

        # Run first T steps to get to synchronization, and the T more to record values 
        for _ in range(T_stabilization): mod1.one_step()
        mod1.r_historic = []
        mod1.theta_historic = []
        for _ in range(t_averages): mod1.one_step()

        print("alpha:", np.around(a,2), "     r=" , np.mean(mod1.r_historic))

        r_averages.append(np.mean(mod1.r_historic))
        r_stds.append(np.std(mod1.r_historic))

        T =  len(mod1.r_historic)
        last_odds = np.mean([mod1.r_historic[i] for i in range(T-length_for_even_ods-1, T-1, 2)])
        last_evens = np.mean([mod1.r_historic[i] for i in range(T-length_for_even_ods, T, 2)])

        last_min =  min(last_odds, last_evens)
        last_max =  max(last_odds, last_evens)

        r_average_odds.append(last_min)
        r_average_evens.append(last_max)



    code = str(np.random.randint(10000))

    plt.plot(alphas, r_averages)
    plt.title("r_averages_" + code + ".png")
    plt.savefig("r_averages_" + code + ".png")
    plt.show()

    plt.plot(alphas, r_stds)
    plt.title("r_stds_" + code + ".png")
    plt.savefig("r_stds_" + code + ".png")
    plt.show()
    
    plt.plot(alphas, r_average_odds)
    plt.plot(alphas, r_average_evens)
    plt.title("r_even_ods_" + code + ".png")

    plt.savefig("r_even_ods_" + code + ".png")
    plt.show()


    df= pd.DataFrame(data = {'alpha': alphas, "r_averages": r_averages, "r_stds": r_stds, 'r_average_odds':r_average_odds, 'r_average_evens':r_average_evens})
    df.to_csv('r_averages_' + code + '.csv')





In [12]:
# N = 10000
# C = 100
# gamma = 3
# maxW = 100
# t_averages = 10
# T_stabilization = 10

# figures_last_r(N, C, gamma, maxW, t_averages, T_stabilization)


In [14]:
class  KuramotoModelVariation(KuramotoModel):
        # DYNAMICS
    # calculates one step of the Kuramoto function  z <-  2 z + (alpha/C) sum( sin(z_j - z))
    def one_step(self): 
                    
        V_cos = self.matrix.dot(self.cos_vector)    ##   timely steps
        V_sin = self.matrix.dot(self.sin_vector)    ##
        
        self.th_vector = self.th_vector % (2 * np.pi)

        self.th_vector = 2*(np.pi - np.abs(self.th_vector - np.pi))
        self.th_vector += (self.alpha / self.C) * (V_sin * self.cos_vector - V_cos * self.sin_vector)
        self.th_vector = self.th_vector % (2 * np.pi)
        
        self.update_r_theta()
        self.r_historic.append(self.r)
        self.theta_historic.append(self.theta)

        return self.r





def init_Kuramoto_invGamma_file_Variation(N, C, gamma, maxW=10, filename = "_", verbose = False):

    edg_lis = read_write_invGamma_graph(N, C, gamma ,maxW, filename = filename, verbose = verbose)
        
    if verbose: print(f"Generating model from edge-list. N={N}, C={C}, g={gamma}")    
    mod =  KuramotoModelVariation(edg_lis)        

    mod.gamma = gamma
    mod.maxW = maxW
    mod.initialization = 'init_Kuramoto_invGamma_file'
    mod.graph_filename = 'invGamma_graph' + filename + f"{N}_{C}_{gamma}_{maxW}.txt"
    
    return mod
